# Trabalhando com arquivos SPM em Pandas

Importação das bibliotecas

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import date, timedelta

%matplotlib inline

Mostrando números de ponto flutuante com separador de milhar e duas casas decimais.

In [2]:
pd.options.display.float_format = '{:,.2f}'.format

Dicionário de dtypes.

Usado para evitar que o engine do pandas tenha que inferir os tipos da tabela, o que pode causar problemas com arquivos csv muito grandes.

In [3]:
_dtype_dict = {
    "Subs. Service Line No.": str,
    "Compute Type": str,
    "Active": str,
    "Usage Line Number": float,
    # "Metered service date": np.datetime64,
    "Sub Account": float,
    "Computed Quantity": float,
    "Existing Entitled Qty": float,
    "Usage Quantity": float,
    "UOM": str,
    "Product": str,
    "Parent Product": str,
    "Net Unit Price": float,
    "SPM Document Number": str,
    "Line Net Amount": float,
    "Bill To Plan Number": float,
    "Consumed Quantity": float,
    "Overage": str,
    "Compute Source": float,
    "Status": str,
    "Available Amt After Computing": float,
    "Is Credit Memo?": str,
    "Spm Partner Credit Amt": float,
    "Bill to Customer": str,
    "Trxn Extension ID": float,
    "Line Net Amount Increased Precision": float,
    "UCM Payment Credit Amt": float,
}

Carregaremos do aquivo csv apenas as colunas da lista abaixo.

In [4]:
cols_list = ['Computed Quantity', 'Metered service date', 'UOM', 'Product', 'Net Unit Price', 'Line Net Amount', 'Bill to Customer']

Abrindo e carregando os arquivos spm.

In [5]:
with open("./BBTS.csv") as csvfile:
    bbts_df = pd.read_csv(csvfile, parse_dates=["Metered service date"], dtype=_dtype_dict, usecols=cols_list)[:-1]

with open("./ATIVOS.csv") as csvfile:
    ativos_df = pd.read_csv(csvfile, parse_dates=["Metered service date"], dtype=_dtype_dict, usecols=cols_list)[:-1]

In [6]:
# Achando o item de maior consumo no dataframe da Ativos
# Corresponde a informação de créditos do Funded Allocation que não foram consumidos
# ativos_df.sort_values(by='Line Net Amount', ascending=False)

Removendo os créditos de funded allocation não consumidos.

In [7]:
ativos_df = ativos_df[ativos_df["UOM"] != "CURRENCY UNIT"]
# ativos_df.drop(ativos_df.loc[ativos_df["UOM"] == "CURRENCY UNIT"].index, inplace=True)

Merge dos dataframes

In [8]:
df = pd.concat([bbts_df, ativos_df])

Vejamos como está o DataFrame

In [9]:
df

,Metered service date,Computed Quantity,UOM,Product,Net Unit Price,Line Net Amount,Bill to Customer
0,2021-07-31,1.00,OCPU PER HOUR,B90572 - Oracle Cloud Infrastructure - Databas...,5.02,5.02,10208161 - Bb Tecnologia E Servicos S.A
1,2021-08-01,1.00,OCPU PER HOUR,B90572 - Oracle Cloud Infrastructure - Databas...,5.02,5.02,10208161 - Bb Tecnologia E Servicos S.A
2,2021-08-02,1.00,OCPU PER HOUR,B90572 - Oracle Cloud Infrastructure - Databas...,5.02,5.02,10208161 - Bb Tecnologia E Servicos S.A
3,2021-07-31,1.00,OCPU PER HOUR,B90572 - Oracle Cloud Infrastructure - Databas...,5.02,5.02,10208161 - Bb Tecnologia E Servicos S.A
4,2021-08-01,1.00,OCPU PER HOUR,B90572 - Oracle Cloud Infrastructure - Databas...,5.02,5.02,10208161 - Bb Tecnologia E Servicos S.A
...,...,...,...,...,...,...,...
41631,2021-08-06,40.00,MBPS PR HR,B92602 - Oracle Cloud Infrastructure - Load Ba...,0.00,0.02,15820392 - Netmanagement Informatica Ltda - Epp
41632,2021-06-08,30.00,MBPS PR HR,B92602 - Oracle Cloud Infrastructure - Load Ba...,0.00,0.01,15820392 - Netmanagement Informatica Ltda - Epp
41633,2021-07-04,30.00,MBPS PR HR,B92602 - Oracle Cloud Infrastructure - Load Ba...,0.00,0.01,15820392 - Netmanagement Informatica Ltda - Epp
41634,2021-06-25,30.00,MBPS PR HR,B92602 - Oracle Cloud Infrastructure - Load Ba...,0.00,0.01,15820392 - Netmanagement Informatica Ltda - Epp


Split coluna 'Product'

A coluna 'Product' do arquivo SPM tem várias informações interessantes, entre elas o part number do produto, sua categoria e o nome

In [10]:
df[['PartNumber', 'Categoria Produto', 'Produto']] = df['Product'].str.split(pat=" - ", n=2, expand=True)

Split da coluna 'Bill to Customer'

In [11]:
df[['Numero Cliente', 'Cliente']] = df['Bill to Customer'].str.split(pat='-', n=1, expand=True)

Se livrando de colunas redundantes

In [12]:
df.drop(labels=['Product', 'Bill to Customer'], axis=1, inplace=True)

Renomeando as colunas

In [13]:
df.rename(columns={"Metered service date": "Data",
                   "Computed Quantity": "Quantidade",
                   "UOM": "Unidade de Medida",
                   "Net Unit Price": "Preco Unitario BRL",
                   "Line Net Amount":"Consumo BRL"},
          inplace=True)

Criando o campo calculado Consumo USD

In [14]:
df['Consumo USD'] = df['Consumo BRL'] / 5.31

Transformando a 'Data' no índice do DataFrame

In [15]:
df.set_index('Data', inplace=True)

Vamos dar uma olhada em como está o nosso DataFrame depois de todas essas transformações

In [16]:
df

,Quantidade,Unidade de Medida,Preco Unitario BRL,Consumo BRL,PartNumber,Categoria Produto,Produto,Numero Cliente,Cliente,Consumo USD
Data,,,,,,,,,,
2021-07-31,1.00,OCPU PER HOUR,5.02,5.02,B90572,Oracle Cloud Infrastructure,Database Cloud Service - Enterprise Edition Ex...,10208161,Bb Tecnologia E Servicos S.A,0.95
2021-08-01,1.00,OCPU PER HOUR,5.02,5.02,B90572,Oracle Cloud Infrastructure,Database Cloud Service - Enterprise Edition Ex...,10208161,Bb Tecnologia E Servicos S.A,0.95
2021-08-02,1.00,OCPU PER HOUR,5.02,5.02,B90572,Oracle Cloud Infrastructure,Database Cloud Service - Enterprise Edition Ex...,10208161,Bb Tecnologia E Servicos S.A,0.95
2021-07-31,1.00,OCPU PER HOUR,5.02,5.02,B90572,Oracle Cloud Infrastructure,Database Cloud Service - Enterprise Edition Ex...,10208161,Bb Tecnologia E Servicos S.A,0.95
2021-08-01,1.00,OCPU PER HOUR,5.02,5.02,B90572,Oracle Cloud Infrastructure,Database Cloud Service - Enterprise Edition Ex...,10208161,Bb Tecnologia E Servicos S.A,0.95
...,...,...,...,...,...,...,...,...,...,...
2021-08-06,40.00,MBPS PR HR,0.00,0.02,B92602,Oracle Cloud Infrastructure,Load Balancer Bandwidth - Mbps Per Hour,15820392,Netmanagement Informatica Ltda - Epp,0.00
2021-06-08,30.00,MBPS PR HR,0.00,0.01,B92602,Oracle Cloud Infrastructure,Load Balancer Bandwidth - Mbps Per Hour,15820392,Netmanagement Informatica Ltda - Epp,0.00
2021-07-04,30.00,MBPS PR HR,0.00,0.01,B92602,Oracle Cloud Infrastructure,Load Balancer Bandwidth - Mbps Per Hour,15820392,Netmanagement Informatica Ltda - Epp,0.00


Lista ordenada dos produtos mais consumidos

In [17]:
 prod_ordem_consumo = df.groupby("Produto")["Consumo BRL"].sum().sort_values(ascending=False).index.to_list()
 prod_ordem_consumo

['Database OCPU - OCPU Per Hour',
 'Compute - Microsoft SQL Standard - OCPU Per Hour',
 'Object Storage - Storage - Gigabyte Storage Capacity per Month',
 'Compute - Windows OS - OCPU Per Hour',
 'FastConnect 10 Gbps - Port Hour',
 'Block Volume Performance - Performance Units Per Gigabyte Per Month',
 'Compute - Virtual Machine Standard - X7 - OCPU Per Hour',
 'Block Volume Storage - Gigabyte Storage Capacity Per Month',
 'Database Cloud Service - Enterprise Edition Extreme Performance - OCPU Per Hour',
 'Database Cloud Service - Enterprise Edition - OCPU Per Hour',
 'Compute - Standard - E3 - OCPU - OCPU Per Hour',
 'Compute - Standard - E3 - Memory - Gigabyte Per Hour',
 'Outbound Data Transfer - Originating in North America, Europe, and UK - Gigabyte Outbound Data Transfer Per Month',
 'Logging - Storage - Gigabyte Log Storage Per Month',
 'Outbound Data Transfer - Originating in APAC, Japan, and South America - Gigabyte Outbound Data Transfer Per Month',
 'Load Balancer Base - Loa

Substituindo o nome dos produtos menos consumidos por 'Outros' para facilitar o agrupamento das informações nos dashboards.

In [18]:
df['Produto Agrupado'] = df['Produto'].replace(prod_ordem_consumo[6:], 'Outros')

In [19]:
df.groupby('Produto Agrupado').sum()

,Quantidade,Preco Unitario BRL,Consumo BRL,Consumo USD
Produto Agrupado,,,,
Block Volume Performance - Performance Units Per Gigabyte Per Month,"946,210.97",19.89,"5,091.95",958.94
Compute - Microsoft SQL Standard - OCPU Per Hour,"19,792.00","2,896.56","23,181.38","4,365.61"
Compute - Windows OS - OCPU Per Hour,"27,513.84","1,226.10","13,355.60","2,515.18"
Database OCPU - OCPU Per Hour,"77,523.80","4,368.78","137,063.13","25,812.27"
FastConnect 10 Gbps - Port Hour,"2,474.00","13,308.64","13,310.12","2,506.61"
Object Storage - Storage - Gigabyte Storage Capacity per Month,"171,739.07",263.37,"13,854.03","2,609.05"
Outros,"824,592.95","5,057.02","23,012.38","4,333.78"


In [20]:
df.groupby(['Produto Agrupado', pd.Grouper(freq='D')]).sum()

Quantidade  \
Produto Agrupado                                   Data                     
Block Volume Performance - Performance Units Pe... 2021-06-01    8,531.98   
                                                   2021-06-02    8,531.98   
                                                   2021-06-03    8,531.98   
                                                   2021-06-04    8,531.98   
                                                   2021-06-05    8,531.98   
...                                                                   ...   
Outros                                             2021-09-08    8,852.49   
                                                   2021-09-09    8,432.11   
                                                   2021-09-10    9,122.13   
                                                   2021-09-11    8,971.44   
                                                   2021-09-12      603.46   

                                                               Preco Unitario BRL  \
Produto Agrupado                                   Data                             
Block Volume Performance - Performance Units Pe... 2021-06-01                0.13   
                                                   2021-06-02                0.13   
                                                   2021-06-03                0.13   
                                                   2021-06-04                0.13   
                                                   2021-06-05                0.13   
...                                                                           ...   
Outros                                             2021-09-08               90.71   
                                                   2021-09-09               90.82   
                                                   2021-09-10               90.95   
                                                   2021-09-11               87.88   
                                                   2021-09-12                1.47   

                                                               Consumo BRL  \
Produto Agrupado                                   Data                      
Block Volume Performance - Performance Units Pe... 2021-06-01        46.08   
                                                   2021-06-02        46.08   
                                                   2021-06-03        46.08   
                                                   2021-06-04        46.08   
                                                   2021-06-05        46.08   
...                                                                    ...   
Outros                                             2021-09-08       381.32   
                                                   2021-09-09       377.11   
                                                   2021-09-10       391.69   
                                                   2021-09-11       377.09   
                                                   2021-09-12        13.93   

                                                               Consumo USD  
Produto Agrupado                                   Data                     
Block Volume Performance - Performance Units Pe... 2021-06-01         8.68  
                                                   2021-06-02         8.68  
                                                   2021-06-03         8.68  
                                                   2021-06-04         8.68  
                                                   2021-06-05         8.68  
...                                                                    ...  
Outros                                             2021-09-08        71.81  
                                                   2021-09-09        71.02  
                                                   2021-09-10        73.76  
                                                   2021-09-11        71.02  
                

Reordenando as colunas

In [21]:
df = df[['Numero Cliente', 'Cliente', 'PartNumber', 'Categoria Produto', 'Produto', 'Produto Agrupado', 'Quantidade', 'Preco Unitario BRL', 'Unidade de Medida', 'Consumo BRL', 'Consumo USD']]
df

,Numero Cliente,Cliente,PartNumber,Categoria Produto,Produto,Produto Agrupado,Quantidade,Preco Unitario BRL,Unidade de Medida,Consumo BRL,Consumo USD
Data,,,,,,,,,,,
2021-07-31,10208161,Bb Tecnologia E Servicos S.A,B90572,Oracle Cloud Infrastructure,Database Cloud Service - Enterprise Edition Ex...,Outros,1.00,5.02,OCPU PER HOUR,5.02,0.95
2021-08-01,10208161,Bb Tecnologia E Servicos S.A,B90572,Oracle Cloud Infrastructure,Database Cloud Service - Enterprise Edition Ex...,Outros,1.00,5.02,OCPU PER HOUR,5.02,0.95
2021-08-02,10208161,Bb Tecnologia E Servicos S.A,B90572,Oracle Cloud Infrastructure,Database Cloud Service - Enterprise Edition Ex...,Outros,1.00,5.02,OCPU PER HOUR,5.02,0.95
2021-07-31,10208161,Bb Tecnologia E Servicos S.A,B90572,Oracle Cloud Infrastructure,Database Cloud Service - Enterprise Edition Ex...,Outros,1.00,5.02,OCPU PER HOUR,5.02,0.95
2021-08-01,10208161,Bb Tecnologia E Servicos S.A,B90572,Oracle Cloud Infrastructure,Database Cloud Service - Enterprise Edition Ex...,Outros,1.00,5.02,OCPU PER HOUR,5.02,0.95
...,...,...,...,...,...,...,...,...,...,...,...
2021-08-06,15820392,Netmanagement Informatica Ltda - Epp,B92602,Oracle Cloud Infrastructure,Load Balancer Bandwidth - Mbps Per Hour,Outros,40.00,0.00,MBPS PR HR,0.02,0.00
2021-06-08,15820392,Netmanagement Informatica Ltda - Epp,B92602,Oracle Cloud Infrastructure,Load Balancer Bandwidth - Mbps Per Hour,Outros,30.00,0.00,MBPS PR HR,0.01,0.00
2021-07-04,15820392,Netmanagement Informatica Ltda - Epp,B92602,Oracle Cloud Infrastructure,Load Balancer Bandwidth - Mbps Per Hour,Outros,30.00,0.00,MBPS PR HR,0.01,0.00


Ordenando pelo ínidce

In [22]:
df = df.sort_index()

Exportando para o Excel

In [23]:
df.to_excel('Consumo.xlsx')